## Imports and initialize vars

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

gemini_model_name = os.getenv("GEMINI_MODEL_FLASH")
print("Gemini Model Name:", gemini_model_name)

## Initialize Google ADK

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

print("✅ ADK components imported successfully.")

In [ ]:
import json
from pathlib import Path

json_path = Path("../data/ep001_remote_work.json")
with json_path.open("r", encoding="utf-8") as f:
    dataset = json.load(f) 
    dataset_json_payload = json.dumps(dataset, indent=2)  # JSON payload available to other cells
    print(f"Loaded JSON from {json_path} (type: {type(dataset).__name__})")
# print(dataset_json_payload)

## Summarizer agent

In [ ]:
summarizer_agent_instruction_template = f"""You are a specialized podcast summarizer agent.
        - Your job is to analyze raw data provided below in json format 
        - Podcast raw data: {dataset_json_payload}.
        - Generate a summary of each episode (with 200–300 words) that captures:
            • Core themes
            • Key discussions
            • Outcomes or opinions shared
        - Provide the output in a markdown table format with columns for 'Core Themes', 'Key Discussions', and 'Outcomes/Opinions'.
        """
# print(summarizer_agent_instruction_template)

In [ ]:
if gemini_model_name is None:
    raise ValueError("Environment variable 'GEMINI_MODEL_FLASH' is not set. Please set it before running this cell.")

summarizer_agent = Agent(
    name="summarizer_agent",
    model=gemini_model_name,
    instruction=summarizer_agent_instruction_template,
)

print("✅ summarizer_agent created.")

In [ ]:
key_insights_agent_instructions_template = f"""You are a specialized podcasts key insights agent.
        - Your job is to analyze raw data provided below in json format 
        - Podcast raw data: {dataset_json_payload}.
        - You need to produce a structured list of:
          •	Top 5 takeaways
          •	Notable quotes (with timestamps)
          • Topics discussed (tag-style labels)
        - Provide the output in a markdown table format with columns for 'Takeaways', 'Quotes', and 'Topics'. 
        """
# print(key_insights_agent_instructions_template)

## Key Insights agent

In [ ]:
key_insights_agent = Agent(
    name="key_insights_agent",
    model="gemini-2.5-flash-lite",
    instruction=key_insights_agent_instructions_template,
    output_key="key_insights_agent_results",
)

print("✅ key_insights_agent created.")

## Fact checking agent


In [ ]:
if gemini_model_name is None:
    raise ValueError("Environment variable 'GEMINI_MODEL_FLASH' is not set. Please set it before running this cell.")

search_agent = Agent(
    model=gemini_model_name,
    name='search_agent',
    instruction="You specialize in crafting concise Google Search queries to support fact-checking requests.",
    tools=[google_search]
)

fact_checker_agent = Agent(
    name="fact_checker_agent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a fact checker agent.
    - Your job is to analyze the summarized podcast data and key insights provided by other agents.
    - Identify factual statements (e.g., “NASA Mars mission launches in 2026”) and verify them using reliable external sources like google search or Wikipedia,
    - Mark each as:
        •	✅ Verified true
        •	⚠️ Possibly outdated/inaccurate
        •	❓ Unverifiable
    - Here is an Example output:
        Claim	Verification	Confidence
        NASA’s Mars mission launches in 2026	True (confirmed by mock NASA DB)	0.92
        Bitcoin will reach $200K by 2025	Unverifiable	0.40""",
    output_key="fact_checker_agent_results",
    tools=[AgentTool(summarizer_agent), AgentTool(key_insights_agent), AgentTool(search_agent)],

)

print("✅ fact_checker_agent created.")

In [ ]:

runner = InMemoryRunner(agent=fact_checker_agent)
response = await runner.run_debug(f"""Use {dataset_json_payload} to analyze the podcast episode data.
                                  - Summarize the episode content.
                                  - Extract key insights including top takeaways, notable quotes, and topics discussed.
                                  - Fact-check significant claims made in the episode using external sources.
                                  - Provide a markdown table summarizing:
                                    • Episode summaries
                                    • Key insights
                                    • Fact-checking results 
                                  - Print the response in human readable format""")